<h1 style="color:orange">Hidden Markov Model (HMM) </h1>

---
<h2 style="color:orange">Markov Model (MM) </h2>

A Markov Model is a stochastic state space model involving random transitions
between states where the probability of the jump is only dependent upon the
current state, rather than any of the previous states. The model is said to
possess the Markov Property and is <span style="color:orange">"memoryless"</span>.
Random Walk models are another familiar example of a Markov Model.


In [1]:
import numpy as np
from ipysketch import Sketch

sk = Sketch('hmm')

In [2]:
sk

Button(description='Edit', style=ButtonStyle())

Output()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x06\x08\x00\x00\x04\x01\x08\x02\x00\x00\x00p\x98\x96…

In [3]:
![gmm](../contents/hmm.png)

zsh:1: number expected


once the Viterbi algorithm has computed the most likely path through the Hidden
Markov Model (HMM), the model can be used to predict future movements.

In an HMM, the state sequence represents the underlying dynamics of the system
being modeled. By using the most likely state sequence, the model can provide a
prediction of the future movements of the system. Specifically, the model can
predict the most likely sequence of future states and observations, given the
current state and observations.

This prediction can be made using the forward algorithm, which computes the
probability of observing a sequence of future observations given the current
state and all previous observations. The forward algorithm can be used to
compute the probability of all possible future state sequences, and the most
likely sequence can be obtained using the Viterbi algorithm.

For example, in a financial time series analysis, an HMM model can be used to
predict the future price movements of a stock or a portfolio of stocks. The
Viterbi algorithm can be used to find the most likely sequence of hidden states
(e.g., market regimes), and the forward algorithm can be used to predict the
most likely sequence of future price movements, given the current market regime.

In summary, once the <p style="color:blue">Viterbi algorithm</p> has computed
the most likely path through the HMM, the model can be used to predict future
movements using the forward algorithm.

In [5]:
# Define the HMM parameters
states = hidden_states = ['H1', 'H2']
obs = observations = ['O1', 'O2', 'O3']
P0 = start_prob = {'H1': 0.6, 'H2': 0.4}
PT = transition_prob = {
    'H1': {'H1': 0.7, 'H2': 0.3},
    'H2': {'H1': 0.4, 'H2': 0.6}
}
PE = emission_prob = {
    'H1': {'O1': 0.1, 'O2': 0.4, 'O3': 0.5},
    'H2': {'O1': 0.6, 'O2': 0.3, 'O3': 0.1}
}

# Define the sequence of observations
obs_seq = ['O1', 'O1', 'O2', 'O2', 'O2', 'O3', 'O1']

# Implement the Viterbi algorithm (to find the most likely sequence of hidden states)
def viterbi(obs_seq, hidden_states, start_prob, transition_prob, emission_prob):
    T = len(obs_seq)
    N = len(hidden_states)

    # Initialize the probability matrix and the backpointer matrix
    prob = np.zeros((T, N))
    backpointer = np.zeros((T, N), dtype=int)

    # Set the initial probabilities
    for s in range(N):
        # prob[0][s] = start_prob[hidden_states[s]] * emission_prob[hidden_states[s]][obs_seq[0]]
        prob[0][s] = P0[states[s]] * PE[states[s]][obs_seq[0]]

    # Calculate the probabilities for each time step
    for t in range(1, T):
        for s in range(N):
            max_prob = 0
            max_state = 0
            for s_prev in range(N):
                curr_prob = prob[t-1][s_prev] * PT[states[s_prev]][states[s]] \
                            * PE[states[s]][obs_seq[t]]
                if curr_prob > max_prob:
                    max_prob = curr_prob
                    max_state = s_prev
            prob[t][s] = max_prob
            backpointer[t][s] = max_state

    # print the probabilities
    for t in range(0, T):
        for s in range(N):
            print("p^{{sample {0}}}_{{{1},{2}}}={3:.6f}, ".format(
                  t+1, states[s], obs_seq[t], prob[t][s]), 
                  end=" ")
        print("\n")

    # Find the path with the highest probability
    max_prob = max(prob[T-1])
    max_state = np.argmax(prob[T-1])
    path = [hidden_states[max_state]]
    for t in range(T-1, 0, -1):
        max_state = backpointer[t][max_state]
        path.insert(0, hidden_states[max_state])

    return path

# Find the most likely sequence of hidden states
path = viterbi(obs_seq, hidden_states, start_prob, transition_prob, emission_prob)
print("Most likely sequence of hidden states:", path)

p^{sample 1}_{H1,O1}=0.060000,  p^{sample 1}_{H2,O1}=0.240000,  

p^{sample 2}_{H1,O1}=0.009600,  p^{sample 2}_{H2,O1}=0.086400,  

p^{sample 3}_{H1,O2}=0.013824,  p^{sample 3}_{H2,O2}=0.015552,  

p^{sample 4}_{H1,O2}=0.003871,  p^{sample 4}_{H2,O2}=0.002799,  

p^{sample 5}_{H1,O2}=0.001084,  p^{sample 5}_{H2,O2}=0.000504,  

p^{sample 6}_{H1,O3}=0.000379,  p^{sample 6}_{H2,O3}=0.000033,  

p^{sample 7}_{H1,O1}=0.000027,  p^{sample 7}_{H2,O1}=0.000068,  

Most likely sequence of hidden states: ['H2', 'H2', 'H1', 'H1', 'H1', 'H1', 'H2']
